In [ ]:
import pandas as pd

# verinin ilk 200 satırını yükle
df = pd.read_csv("../../data/final_dataset.csv", nrows=200)

In [3]:
# Verinin ilk 5 satırını göstererek sütunları ve içeriği gör
df.head()

,url,title,summary,publish_date,article_text
0,https://www.anahaberajansi.com.tr/gunluk-burc-...,Günlük burç yorumları: 15 Mayıs 2025 Perşembe,Güneş’in bugün Mars ile paralel açısı kararlıl...,NaN,Güneş’in bugün Mars ile paralel açısı kararlıl...
1,https://www.anahaberajansi.com.tr/2025-yili-du...,2025 yılı düğünlerinde nostalji trendi,Retro masa lambaları,NaN,Retro masa lambaları\nDüğün masalarını süsleye...
2,https://www.anahaberajansi.com.tr/ozenle-korun...,Özenle korunan 7 bin plaklık müzik mirası,İstanbul Maltepe’de yaşayan 57 yaşındaki müzik...,NaN,İstanbul Maltepe’de yaşayan 57 yaşındaki müzik...
3,https://www.anahaberajansi.com.tr/titanikin-ba...,Titanik’in batmadan önce bir yolcunun yazdığı ...,Yayınlandığında tüm dünyada adeta izlenme reko...,NaN,Yayınlandığında tüm dünyada adeta izlenme reko...
4,https://www.anahaberajansi.com.tr/konyadaki-ge...,Konya’daki Gevele Kalesi turizme kazandırılacak,"Merkez Selçuklu ilçesinde bulunan Helenistik, ...",NaN,"Merkez Selçuklu ilçesinde bulunan Helenistik, ..."


In [5]:
# Veri setinin genel yapısını, sütun tiplerini ve boş olmayan veri sayısını gör
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   url           200 non-null    object 
 1   title         200 non-null    object 
 2   summary       200 non-null    object 
 3   publish_date  0 non-null      float64
 4   article_text  200 non-null    object 
dtypes: float64(1), object(4)
memory usage: 7.9+ KB


In [7]:
# Hangi sütunda kaç tane boş veri olduğunu kontrol et
df.isnull().sum()

url               0
title             0
summary           0
publish_date    200
article_text      0
dtype: int64

In [ ]:
import re
import nltk
from nltk.corpus import stopwords

# Gerekli NLTK verilerini indir
nltk.download('stopwords')

# Türkçe stop words listesini al
turkish_stop_words = set(stopwords.words('turkish'))

def preprocess_turkish_text(text):
    # 0. Güvenlik: Eğer veri string değilse boş metin döndür
    if not isinstance(text, str):
        return ""
    
    # 1. HTML etiketlerini temizle
    text = re.sub(r'<.*?>', '', text)
    
    # 2. URL'leri ve e-postaları temizle
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # 3. Sadece harfleri ve boşlukları bırak (Türkçe karakterleri koru)
    # Noktalama işaretlerini ve sayıları kaldırır.
    text = re.sub(r'[^a-zA-ZçÇğĞıİöÖşŞüÜ\s]', '', text)
    
    # 4. Türkçe'ye özel küçük harfe çevirme (I -> ı)
    text = text.replace('I', 'ı').lower()
    
    # 5. Fazla boşlukları temizle (birden fazla boşluğu tek boşluğa indir)
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 6. Stop words temizliği
    words = text.split() # Cümleyi kelimelere ayır
    filtered_words = [word for word in words if word not in turkish_stop_words]
    text = ' '.join(filtered_words) # Temiz kelimeleri tekrar birleştir
    
    return text



[nltk_data] Downloading package stopwords to /home/juan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# DataFrame'den ilk haberi al
sample_article = df['article_text'].iloc[0]

cleaned_article = preprocess_turkish_text(sample_article)

print("--- ORİJİNAL HABER ---")
print(sample_article[:500])
print("\n" + "="*50 + "\n")
print("--- TEMİZLENMİŞ HABER ---")
print(cleaned_article[:500])

--- ORİJİNAL HABER ---
Güneş’in bugün Mars ile paralel açısı kararlılığımızı güçlendirir ve tepkilerimizi hızlandırır. Arzularımız güçlüdür ve harekete geçmeye hazırız. Bu geçiş, enerjiyi ve girişimciliği artırır. İrade sahibiyiz, doğrudan ve motiveyiz; ne istediğimizin ya da neye inandığımızın arkasında durmaya hazırız. Bu rekabetçi bir etkidir ve acelecilikten kaçınarak proaktif olma zamanıdır. Bugün Yay burcundaki Ay, coşkulu, kendine güvenen ve geleceğe dönük bir enerji getirir. Yine de, Güneş ile Jüpiter arasında


--- TEMİZLENMİŞ HABER ---
güneşin bugün mars paralel açısı kararlılığımızı güçlendirir tepkilerimizi hızlandırır arzularımız güçlüdür harekete geçmeye hazırız geçiş enerjiyi girişimciliği artırır i̇rade sahibiyiz doğrudan motiveyiz istediğimizin neye inandığımızın arkasında durmaya hazırız rekabetçi bir etkidir acelecilikten kaçınarak proaktif olma zamanıdır bugün yay burcundaki ay coşkulu kendine güvenen geleceğe dönük bir enerji getirir yine güneş jüpiter arasındaki y

In [14]:
print("Örneklem (200 satır) temizleniyor...")
df['cleaned_article'] = df['article_text'].apply(preprocess_turkish_text)
df['cleaned_summary'] = df['summary'].apply(preprocess_turkish_text)
print("Temizleme tamamlandı!")

Örneklem (200 satır) temizleniyor...
Temizleme tamamlandı!


In [15]:
df[['article_text', 'cleaned_article', 'summary', 'cleaned_summary']].head()

,article_text,cleaned_article,summary,cleaned_summary
0,Güneş’in bugün Mars ile paralel açısı kararlıl...,güneşin bugün mars paralel açısı kararlılığımı...,Güneş’in bugün Mars ile paralel açısı kararlıl...,güneşin bugün mars paralel açısı kararlılığımı...
1,Retro masa lambaları\nDüğün masalarını süsleye...,retro masa lambaları düğün masalarını süsleyen...,Retro masa lambaları,retro masa lambaları
2,İstanbul Maltepe’de yaşayan 57 yaşındaki müzik...,i̇stanbul maltepede yaşayan yaşındaki müzik ya...,İstanbul Maltepe’de yaşayan 57 yaşındaki müzik...,i̇stanbul maltepede yaşayan yaşındaki müzik ya...
3,Yayınlandığında tüm dünyada adeta izlenme reko...,yayınlandığında dünyada adeta izlenme rekorlar...,Yayınlandığında tüm dünyada adeta izlenme reko...,yayınlandığında dünyada adeta izlenme rekorlar...
4,"Merkez Selçuklu ilçesinde bulunan Helenistik, ...",merkez selçuklu ilçesinde bulunan helenistik r...,"Merkez Selçuklu ilçesinde bulunan Helenistik, ...",merkez selçuklu ilçesinde bulunan helenistik r...
